In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
# %cd /content/drive/My Drive/Research Project

In [3]:
import pandas as pd
import numpy as np
import os
# import src
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
# pip install -U mittens

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from statistics import mean, stdev, median, mode
# With PCA
from sklearn.decomposition import PCA

from sklearn.feature_extraction.text import CountVectorizer
# SVM
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
# KNN
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

from mittens import GloVe, Mittens

In [6]:
# pip install autocorrect

In [7]:

import nltk
import string
from nltk.tokenize import word_tokenize
import random
import pickle
from nltk.corpus import stopwords

from autocorrect import Speller
# from pycontractions import Contractions

# from spellchecker import SpellChecker

import re
from nltk.corpus import wordnet
from nltk.stem.wordnet import WordNetLemmatizer 

from hyperopt import fmin, tpe, hp

In [8]:
# nltk.download('all')

In [9]:
from numpy import asarray
from numpy import zeros
from sklearn.utils import shuffle
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score

In [10]:
# nltk.download('all')

In [11]:
# # Load your favorite word2vec model
# cont = Contractions('https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz')
# text = "we're"
# text = list(cont.expand_texts([text], precise=True))[0]

In [12]:
def conv_dataframes(pos_path, neg_path):
    with open(pos_path,'r',encoding='latin1') as f:
        data_p = f.readlines()
#     print(data_p[11])
    with open(neg_path,'r',encoding='latin1') as f:
        data_n = f.readlines()
    pos_data = shuffle(pd.DataFrame(data_p, columns = ["Doc Text"]))
#     pos_data['Sentiment'] = 1
#     pos_data.columns = ["Doc Text", "Sentiment"]
    neg_data = shuffle(pd.DataFrame(data_n, columns = ["Doc Text"]))
#     neg_data['Sentiment'] = -1
#     neg_data.columns = ["Doc Text", "Sentiment"]
    return pos_data, neg_data


## The code for pos tagging and lemmatize sentence is fron the following link:
 ### https://medium.com/@gaurav5430/using-nltk-for-lemmatizing-sentences-c1bfff963258

In [13]:
# function to convert nltk tag to wordnet tag
def nltk_tag_to_wordnet_tag(nltk_tag):
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:          
        return None

In [14]:
def lemmatize_sentence(sentence):
    #tokenize the sentence and find the POS tag for each token
    nltk_tagged = nltk.pos_tag(nltk.word_tokenize(sentence))  
    #tuple of (token, wordnet_tag)
    wordnet_tagged = map(lambda x: (x[0], nltk_tag_to_wordnet_tag(x[1])), nltk_tagged)
#     print(wordnet_tagged)
    lemmatized_sentence = []
    for word, tag in wordnet_tagged:
        if tag is None:
            #if there is no available tag, append the token as is
            lemmatized_sentence.append(word)
        else:        
            #else use the tag to lemmatize the token
            lemmatized_sentence.append(lemmatizer.lemmatize(word, tag))
#         if tag is not None:
#             lemmatized_sentence.append(lemmatizer.lemmatize(word, tag)) 

    return " ".join(lemmatized_sentence)

In [15]:
# print(lemmatizer.lemmatize("I am loving it")) #I am loving it
# print(lemmatizer.lemmatize("loving")) #loving
# print(lemmatizer.lemmatize("loving", "v")) #love
# print(lemmatize_sentence("I am loving it")) #I be love it

In [16]:
# obama_train_pr.to_csv(os.path.join(data_path, 'Obama Training1 Data.csv'))
# romney_train_pr.to_csv(os.path.join(data_path, 'Romney Training1 Data.csv'))

In [17]:
def replaceTwoOrMore(s):
    #look for 2 or more repetitions of character
    pattern = re.compile(r"(.)\1{1,}", re.DOTALL) 
    return pattern.sub(r"\1\1", s)

replaceTwoOrMore('Goooddd')

'Goodd'

In [18]:
re.sub(r"\b(?:a{0,2}h{1,2}a{0,2}){2,}h?\b", "haha", "hahahahahahahhhahaha")

'haha'

In [19]:
re.sub(r"\b(?:o{0,2}l{1,2}o{0,2}){2,}l?\b", "lol", "lolololl")

'lol'

In [20]:
str = '#Obama'

str.replace('\S+', ' ')

'#Obama'

In [21]:
def text_preprocessing(data):
    
    # This method replaces two or more consecutive letters with the same character to something shorter. For example, gooooooood becomes good.
    def replaceTwoOrMore(s):
        #look for 2 or more repetitions of character
        pattern = re.compile(r"(.)\1{1,}", re.DOTALL) 
        return pattern.sub(r"\1\1", s)

    # This method converts camel cased words into space delimited words.
    # For example: ThisIsASentence will be changed to This Is A Sentence
    def convertCamelCase(word):
        return re.sub("([a-z])([A-Z])","\g<1> \g<2>",word)

    # Read a flat file containing some abbreviations and their expansions in pipe separated format
    # Use these abbreviations to replace text in the tweets as part of Preprocessing
    
    def readAbbrFile(abb_path):
        global abbr_dict
        abbr_dict ={}
        f = open(abb_path)
        lines = f.readlines()
        f.close()
        for i in lines:
            tmp = i.split('|')
            abbr_dict[tmp[0]] = tmp[1]

        return abbr_dict
  
    # This function checks the dictionary containing abbreviations and their meanings as (key,value) pairs
    # and replaces the key with the corresponding value
    def replaceAbbr(s):
#         temp =[]
#         for word in s.split():
#             if word.lower() in abbr_dict.keys():
# #                 print('t')                
#                 temp.append(abbr_dict[word.lower()])
#             else:
#                 temp.append(word)
        temp = " ".join([abbr_dict[word.lower()] if word.lower() in abbr_dict.keys() else word for word in s.split()])
        return temp
    #end

    def readcontractions(contra_path):
        global contra_dict
        contra_dict ={}
        f = open(contra_path)
        lines = f.readlines()
        f.close()
        for i in lines:
            try: 
                tmp = i.replace('"', '').replace(',', '').replace('\n', ' ').split(':')
                contra_dict[tmp[0]] = tmp[1]
            except:
                print(tmp)
                print(z)

        return contra_dict
    
    # This function checks the dictionary containing abbreviations and their meanings as (key,value) pairs
    # and replaces the key with the corresponding value
    def replacecontra(s):
        temp = " ".join([contra_dict[word.lower()] if word.lower() in contra_dict.keys() else word for word in s.split()])
        return temp
    #end    
    

    abb_path = os.path.join(data_path,"abbrevations.txt")
    abbr_dict = readAbbrFile(abb_path)
    
    contra_path = os.path.join(data_path,"contractions.txt")
    contra_dict = readcontractions(contra_path)
    data["Doc Text"] = data["Doc Text"].str.replace('[^\x00-\x7f]', ' ') # remove non ascii characters
    
    data["Doc Text"] = data["Doc Text"].apply(lambda x: x.lower())
    data["Doc Text"] = data["Doc Text"].apply(lambda x: replaceAbbr(x))
    data["Doc Text"] = data["Doc Text"].apply(lambda x: replacecontra(x))

    data["Doc Text"] = data["Doc Text"].str.replace('\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*',' ') #remove URL
    data["Doc Text"] = data["Doc Text"].str.replace('(\s)@\w+', ' ') #remove usernames
    data["Doc Text"] = data["Doc Text"].str.replace('@\w+', ' ') #remove usernames
    data["Doc Text"] = data["Doc Text"].str.replace('<[^<]+?>', ' ') #remove HTML tags
    data["Doc Text"] = data["Doc Text"].str.replace('[<>!#@$:.,%\?-_]+', ' ') #remove punctuation and special characters
    # data["Doc Text"] = data["Doc Text"].str.replace('(#\S+)', ' ') # to remove words with hashtags #Obama


    
    data["Doc Text"] = data["Doc Text"].str.replace('\d+', ' ') # removing the words with more than 1 digit
    data["Doc Text"] = data["Doc Text"].str.replace('\n\n', ' ')
    data["Doc Text"] = data["Doc Text"].str.replace('\n', ' ') # removing new line characters
    data["Doc Text"] = data["Doc Text"].str.replace('[^\w\s]',' ')
#     data["Doc Text"] = data["Doc Text"].str.replace('\s+[a-zA-Z]\s+',' ')
#     data["Doc Text"] = data["Doc Text"].str.replace('\^[a-zA-Z]\s+',' ')
    data["Doc Text"] = data["Doc Text"].str.replace('\s+',' ')  

    
    data["Doc Text"] = data["Doc Text"].apply(lambda x: replaceTwoOrMore(x))
    data["Doc Text"] = data["Doc Text"].apply(lambda x: convertCamelCase(x))

    # Remove stop words from text
    data["Doc Text"] = data["Doc Text"].apply(lambda x: " ".join([word for word in x.split() if word not in stop_list]))


    data["Doc Text"] = data["Doc Text"].apply(lambda x: " ".join([re.sub(r"\b(?:a{0,2}h{1,2}a{0,2}){2,}h?\b", "haha", word) for word in x.split()])) # REPLACING HAHAHAH WITH HAHA
    data["Doc Text"] = data["Doc Text"].apply(lambda x: " ".join([re.sub(r"\b(?:o{0,2}l{1,2}o{0,2}){2,}l?\b", "lol", word) for word in x.split()])) # REPLACING lOLOLOL WITH LOL
                                                  
#     data["Doc Text"] = data["Doc Text"].apply(lambda x: " ".join([lemmatizer.lemmatize(y) for y in x.split()]))
    data["Doc Text"] = data["Doc Text"].apply(lambda x: lemmatize_sentence(x))

    return data

In [22]:
st = 'k k  k'
st.replace('\s+','')
st.replace('\n\n', '')
st.replace('[^\w\s]','')
st.replace('\s+[a-zA-Z]\s+','')
st.replace('\^[a-zA-Z]\s+','')

'k k  k'

In [23]:
def create_vocab(data1, data2):
    temp1 = text_preprocessing(data1)
    temp2 = text_preprocessing(data2)
#     temp_pos2 = np.asarray([word_tokenize(re.sub(r"\b[a-zA-Z]\b", " ",i)) for i in temp_pos['Doc Text']])
#     temp_neg2 = np.asarray([word_tokenize(re.sub(r"\b[a-zA-Z]\b", " ",i)) for i in temp_neg['Doc Text']]) 
#     temp_pos["Doc Text Tokens"] = temp_pos2 
#     temp_neg["Doc Text Tokens"] = temp_neg2
    return temp1, temp2

In [24]:
data_path = r'C:\Users\kalya\OneDrive - University of Illinois at Chicago\!UIC\!Semesters\3rd Sem\CS 583 Data Mining and Text Mining\Research Project\Data'

In [25]:

trainFile = os.path.join(data_path,"training-Obama-Romney-tweets.xlsx")
obama_train1_temp = pd.read_excel(trainFile, sheet_name = 'Obama', header = None, skiprows =[0,1], usecols= [3,4], names =['Doc Text', 'Sentiment'])
romney_train1_temp = pd.read_excel(trainFile, sheet_name = 'Romney', header = None, skiprows =[0,1], usecols= [3,4], names =['Doc Text', 'Sentiment'])



In [26]:
obama_train1_temp.head()

,Doc Text,Sentiment
0,"Kirkpatrick, who wore a baseball cap embroider...",0
1,Question: If <e>Romney</e> and <e>Obama</e> ha...,2
2,#<e>obama</e> debates that Cracker Ass Cracker...,1
3,RT @davewiner Slate: Blame <e>Obama</e> for fo...,2
4,@Hollivan @hereistheanswer Youre missing the ...,0


In [27]:
romney_train1_temp.head()

,Doc Text,Sentiment
0,Insidious!<e>Mitt Romney</e>'s Bain Helped Phi...,-1
1,Senior <e>Romney</e> Advisor Claims <e>Obama</...,2
2,.@WardBrenda @shortwave8669 @allanbourdius you...,-1
3,<e>Mitt Romney</e> still doesn't <a>believe</a...,-1
4,<e>Romney</e>'s <a>tax plan</a> deserves a 2nd...,-1


## Removing datapoints with mixed sentiment

In [28]:
obama_train1 = obama_train1_temp[obama_train1_temp['Sentiment'] .isin((1,-1,0))]
obama_train1.head()

,Doc Text,Sentiment
0,"Kirkpatrick, who wore a baseball cap embroider...",0
2,#<e>obama</e> debates that Cracker Ass Cracker...,1
4,@Hollivan @hereistheanswer Youre missing the ...,0
6,I was raised as a Democrat left the party yea...,-1
7,The <e>Obama camp</e> can't afford to lower ex...,0


In [29]:
romney_train1 = romney_train1_temp[romney_train1_temp['Sentiment'] .isin((1,-1,0))]
romney_train1.head()

,Doc Text,Sentiment
0,Insidious!<e>Mitt Romney</e>'s Bain Helped Phi...,-1
2,.@WardBrenda @shortwave8669 @allanbourdius you...,-1
3,<e>Mitt Romney</e> still doesn't <a>believe</a...,-1
4,<e>Romney</e>'s <a>tax plan</a> deserves a 2nd...,-1
5,Hope <e>Romney</e> debate prepped w/ the same ...,1


### Dropping Missing Values

In [30]:
obama_train1 = obama_train1.dropna()
romney_train1 = romney_train1.dropna()

## Preprocessing the data

In [31]:
# hm_lines = 5331

# tokenizer = nltk.RegexpTokenizer(r"\w+")
# spell = Speller(lang='en')
# # spell = SpellChecker()

# stop_words = set(stopwords.words('english'))
stop_list = stopwords.words('english')
stop_list.extend(['rt', 'retweet', 'e'])

lemmatizer = WordNetLemmatizer() 

In [32]:
obama_train1_pr, romney_train1_pr = create_vocab(obama_train1, romney_train1)

In [33]:
obama_train1_pr.head()

,Doc Text,Sentiment
0,kirkpatrick wear baseball cap embroider obama ...,0
2,obama debate cracker as cracker tonight tune t...,1
4,miss point afraid understand big picture dont ...,0
6,raise democrat leave party year ago lifetime n...,-1
7,obama camp can not afford low expectation toni...,0


In [34]:
obama_train1_pr['Sentiment'] = obama_train1_pr['Sentiment'].apply(lambda x: 'Positive' if x == 1 else ('Negative' if x == -1 else 'Neutral'))

In [35]:
obama_train1_pr.head()

,Doc Text,Sentiment
0,kirkpatrick wear baseball cap embroider obama ...,Neutral
2,obama debate cracker as cracker tonight tune t...,Positive
4,miss point afraid understand big picture dont ...,Neutral
6,raise democrat leave party year ago lifetime n...,Negative
7,obama camp can not afford low expectation toni...,Neutral


In [36]:
romney_train1_pr.head()

,Doc Text,Sentiment
0,insidious mitt romney bain help philip morris ...,-1
2,mean like romney cheat primary,-1
3,mitt romney still believe black president,-1
4,romney tax plan deserve nd look secret one dif...,-1
5,hope romney debate prepped people last time,1


In [37]:
romney_train1_pr['Sentiment'] = romney_train1_pr['Sentiment'].apply(lambda x: 'Positive' if x == 1 else ('Negative' if x == -1 else 'Neutral'))

In [38]:
romney_train1_pr.head()

,Doc Text,Sentiment
0,insidious mitt romney bain help philip morris ...,Negative
2,mean like romney cheat primary,Negative
3,mitt romney still believe black president,Negative
4,romney tax plan deserve nd look secret one dif...,Negative
5,hope romney debate prepped people last time,Positive



## Saving Processed Data

In [39]:
obama_train1_pr.to_csv(os.path.join(data_path, 'Obama Training1 Data.csv'))
romney_train1_pr.to_csv(os.path.join(data_path, 'Romney Training1 Data.csv'))

In [40]:
obama_train_pr, obama_val_pr =  train_test_split(obama_train1_pr, test_size = 0.2)
romney_train_pr, romney_val_pr =  train_test_split(romney_train1_pr, test_size = 0.2)

In [41]:
obama_train_pr.to_csv(os.path.join(data_path, 'Obama Training Data.csv'))
romney_train_pr.to_csv(os.path.join(data_path, 'Romney Training Data.csv'))

obama_val_pr.to_csv(os.path.join(data_path, 'Obama Validation Data.csv'))
romney_val_pr.to_csv(os.path.join(data_path, 'Romney Validation Data.csv'))

In [42]:
# data_path = r'/content/drive/My Drive/Research Project/Data'

### Combining two datasets

In [43]:
train1_pr = shuffle(pd.concat([obama_train1_pr, romney_train1_pr]))
train1_pr.head()

,Doc Text,Sentiment
5500,debate job come back obama get as make corp ta...,Negative
5175,romney romney abrasive,Negative
1436,sicken obama war woman tom halloran,Negative
3898,mitt romney business genius speculator play bi...,Negative
4032,day election day ready cheer president obama t...,Positive


In [44]:
train1_pr = train1_pr.dropna()
# romney_train_pr_df = romney_train_pr_df.dropna()

In [45]:
train1_pr.to_csv(os.path.join(data_path, 'Combined Training1 Data.csv'))

In [46]:
train_pr, val_pr =  train_test_split(train1_pr, test_size = 0.2)

In [47]:
train_pr.to_csv(os.path.join(data_path, 'Combined Training Data.csv'))

val_pr.to_csv(os.path.join(data_path, 'Combined Validation Data.csv'))


In [1]:
pip install pyrouge

  Created wheel for pyrouge: filename=pyrouge-0.1.3-py3-none-any.whl size=191615 sha256=95cfcd7ba8ea0dfabf1d901b0bed6832cbf2a8bd552cf15fb886ea3a846e468b
  Stored in directory: c:\users\kalya\appdata\local\pip\cache\wheels\00\85\fd\ccd28e53c9f6a691e6ea96050a0cad95f9a4a6361269d765ca
Successfully built pyrouge
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install py-rouge

Note: you may need to restart the kernel to use updated packages.
